In [1]:
import math
import random
import matplotlib.pyplot as plt


In [13]:
pip install pygame

  Obtaining dependency information for pygame from https://files.pythonhosted.org/packages/d2/55/ca3eb851aeef4f6f2e98a360c201f0d00bd1ba2eb98e2c7850d80aabc526/pygame-2.6.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB 163.8 kB/s eta 0:01:05
   ---------------------------------------- 0.0/10.6 MB 217.9 kB/s eta 0:00:49
   ---------------------------------------- 0.1/10.6 MB 302.7 kB/s eta 0:00:35
   ---------------------------------------- 0.1/10.6 MB 409.6 kB/s eta 0:00:26
    --------------------------------------- 0.1/10.6 MB 473.7 kB/s eta 0:00:23
    --------------------------------------- 0.2/1

In [31]:
import pygame
import math
import random
import time

# Node class representing each point in the tree
class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.parent = None
        self.cost = 0.0

class RRTStar:
    def __init__(self, start, goal, obstacle_list, x_range, y_range, step_size=10, max_iter=1000, radius=30):
        self.start = Node(*start)
        self.goal = Node(*goal)
        self.obstacle_list = obstacle_list
        self.x_range = x_range
        self.y_range = y_range
        self.step_size = step_size
        self.max_iter = max_iter
        self.radius = radius
        self.nodes = [self.start]
        pygame.init()
        self.screen = pygame.display.set_mode((x_range[1], y_range[1]))
        pygame.display.set_caption("RRT* Pathfinding Visualization")
        self.clock = pygame.time.Clock()
        self.screen.fill((255, 255, 255))
    
    def draw_circle(self, x, y, radius, color):
        pygame.draw.circle(self.screen, color, (int(x), int(y)), int(radius))

    def draw_line(self, x1, y1, x2, y2, color):
        pygame.draw.line(self.screen, color, (int(x1), int(y1)), (int(x2), int(y2)))

    def distance(self, node1, node2):
        return math.sqrt((node1.x - node2.x) ** 2 + (node1.y - node2.y) ** 2)

    def get_random_node(self):
        x = random.uniform(self.x_range[0], self.x_range[1])
        y = random.uniform(self.y_range[0], self.y_range[1])
        return Node(x, y)

    def nearest_node(self, random_node):
        return min(self.nodes, key=lambda node: self.distance(node, random_node))

    def is_collision_free(self, node1, node2):
        for (ox, oy, size) in self.obstacle_list:
            steps = int(max(abs(node2.x - node1.x), abs(node2.y - node1.y)))
            for i in range(steps):
                x = node1.x + (node2.x - node1.x) * i / steps
                y = node1.y + (node2.y - node1.y) * i / steps
                if math.hypot(ox - x, oy - y) <= size:
                    return False
        return True

    def generate_path(self):
        for _ in range(self.max_iter):
            random_node = self.get_random_node()
            nearest_node = self.nearest_node(random_node)
            theta = math.atan2(random_node.y - nearest_node.y, random_node.x - nearest_node.x)
            new_node = Node(nearest_node.x + self.step_size * math.cos(theta), nearest_node.y + self.step_size * math.sin(theta))
            new_node.cost = nearest_node.cost + self.distance(nearest_node, new_node)

            if self.is_collision_free(nearest_node, new_node):
                new_node.parent = nearest_node
                self.nodes.append(new_node)
                self.dynamic_draw(new_node)

                if self.distance(new_node, self.goal) <= self.step_size:
                    self.goal.parent = new_node
                    self.nodes.append(self.goal)
                    return self.extract_path()
        return None

    def extract_path(self):
        path = []
        node = self.goal
        while node is not None:
            path.append((node.x, node.y))
            node = node.parent
        return path[::-1]

    def dynamic_draw(self, node):
        self.screen.fill((255, 255, 255))
        for ox, oy, size in self.obstacle_list:
            self.draw_circle(ox, oy, size, (255, 0, 0))  # Obstacles in red
        for n in self.nodes:
            if n.parent:
                self.draw_line(n.x, n.y, n.parent.x, n.parent.y, (0, 255, 0))  # Tree edges in green
        self.draw_circle(self.start.x, self.start.y, 5, (0, 0, 255))  # Start in blue
        self.draw_circle(self.goal.x, self.goal.y, 5, (255, 0, 0))  # Goal in red
        pygame.display.update()
        self.clock.tick(30)  # Control frame rate

    def draw_final_path(self, path):
        for i in range(len(path) - 1):
            self.draw_line(path[i][0], path[i][1], path[i + 1][0], path[i + 1][1], (0, 0, 255))  # Final path in blue
        pygame.display.update()

if __name__ == '__main__':
    start = (10, 10)
    goal = (400, 400)
    obstacle_list = [(100, 100, 20), (200, 300, 30), (350, 150, 40)]  # Obstacles as (x, y, radius)
    x_range = (0, 500)
    y_range = (0, 500)

    rrt_star = RRTStar(start, goal, obstacle_list, x_range, y_range)
    path = rrt_star.generate_path()

    if path:
        print("Path found:", path)
        rrt_star.draw_final_path(path)
        time.sleep(5)
    else:
        print("No path found")
    pygame.quit()


Path found: [(10, 10), (18.33766303337398, 15.521175159502643), (24.704582104691006, 23.232358047964016), (29.914033880687168, 31.768259417683), (35.596921362048356, 39.996551456386314), (44.03056644448681, 45.36997035160207), (52.52175859118562, 50.65198284002354), (61.085929943669335, 55.81482789341197), (68.1225703592003, 62.92015629275541), (77.72667790897584, 60.134277944308685), (87.72665272739403, 60.11183621972584), (97.48441307208718, 62.29955490720232), (103.54829746834359, 70.25124064224228), (112.66317393425064, 74.36451512995326), (122.64695961986274, 74.93374564895828), (123.07686238705351, 84.92450055590909), (129.54393833859558, 92.55188075077196), (139.54358798672473, 92.63558806628468), (147.82334293700566, 98.24323088330667), (157.77114316075046, 99.26365761191741), (165.11155573785484, 106.05470636729294), (174.2967163059915, 102.10086958037985), (184.0060101855731, 99.70720871051556), (193.53675042583936, 96.67983518341828), (198.44264014389825, 105.3937462498823),